# Senior Cubers Worldwide - Weekly Competition

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/scw-comp/

## Initialisation

Basic approach to determine the project directory

In [1]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

## Supported Events

Supported events

In [2]:
events = \
{
    '2x2x2': '2x2x2',
    '3x3x3':'3x3x3',
    '4x4x4': '4x4x4',
    '5x5x5': '5x5x5',
    '6x6x6':'6x6x6',
    '7x7x7': '7x7x7',
    'oh': '3x3x3 One-Handed',
    'mega': 'Megaminx',
    '3bld': '3x3x3 Blindfolded',
    '4bld': '4x4x4 Blindfolded',
    '5bld': '5x5x5 Blindfolded',
    'fmc': '3x3x3 Fewest Moves'
}

## Name Mappings

Function to ensure a standard name

In [3]:
nameMap = {
    'Goho Choi': 'Go-ho Choi',
    'Rob Summerfield': 'Robert Summerfield',
    'Shawn Boucke': 'Shawn Boucké',
    'Tim Salay': 'Timothy Salay'
}

def getCorrectName(name):
    '''Handle people who regularly enter their name differently'''
    
    if name in nameMap:
        return nameMap[name]
    else:
        return name

## Formatting Functions

Functions to convert results to and from seconds or display an age category

In [4]:
def numSeconds(value):
    '''Convert float or string to number of seconds - e.g. 1:05.31 returns 65.31'''   
    
    if isinstance(value, float):
        return round(value, 2)
    else:
        value = value.replace(',', '.')
        if ':' in value:
            parts = value.split(':')
            return round(int(parts[0]) * 60 + float(parts[1]), 2)
        elif '.' in value:
            return round(float(value), 2)
        elif 'n/a' in value.lower():
            return 0
        elif 'dnf' in value.lower():
            return -1
        elif 'dns' in value.lower():
            return -2
        else:
            return None


def formatResult(value, eventName):
    '''Convert number of seconds to displayable time - e.g. 65.31 returns 1:05.31'''
    
    if value is not None:
        if value > 0:
            if eventName == 'fmc':
                return '{:d}'.format(int(value))
            else:
                if value > 60:
                    return '{:d}:{:05.2f}'.format(int(value // 60), value - int(value // 60) * 60)
                else:
                    return '{:.2f}'.format(value)
        else:
            if value == 0:
                return '-'
            elif value == -1:
                return 'DNF'
            elif value == -2:
                return 'DNS'
            else:
                return '?'
    else:
        return ''


def interpretAge(age):
    '''Interpret age, however it is written and return an integer'''
    
    age = age.lower()
    if 'under' in age:
        age = int(age.replace('under', '').replace(' ', '')) - 10
    elif 'over' in age:
        age = int(age.replace('over', '').replace(' ', ''))
    elif '<' in age:
        age = int(age.replace('<', '').replace(' ', '')) - 10
    elif '>' in age:
        age = int(age.replace('>', '').replace(' ', ''))
    else:
        age = int(age.replace('+', ''))
        
    return age

    
def formatAge(age):
    '''Format age for report'''
    
    if age < 40:
        return '<{}'.format(age + 10)
    else:
        return '{}+'.format(age)


def formatFacebookLink(link):
    '''Change mobile links to regular Facebook link'''
    
    if link:
        if '//m.' in link:
            link = link.replace('//m.', '//www.')
        if '?view=permalink&id=' in link:
            link = link.replace('?view=permalink&id=', '/permalink/')
        if '?' in link:
            link = link[:link.find('?')]
        if not link.endswith('/'):
            link = link + '/'

    return link

## Generic Class

Generic class to ensure that all custom classes are printable

In [5]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Spreadsheet Class

Class to parse result spreadsheets

In [6]:
from xlrd import open_workbook
from datetime import datetime

import csv

class Spreadsheet(Printable):
    def __init__(self, xlsx):
        self.xlsx = xlsx
        self.date = os.path.basename(os.path.dirname(xlsx))
        self.events = {}

      
    def processLegacySheet(self, sheet):
        '''Process a sheet which was filled in by participants'''

        eventName = sheet.name.lower()
        columnNames = []
        rows = []

        for rowNo in range(sheet.nrows):

            # Process header row
            if rowNo == 2:
                for colNo in range(sheet.ncols):
                    value = sheet.cell(rowNo, colNo).value.strip()
                    
                    # Simple string matches
                    if value in ('Name', 'Age', 'Result'):
                        columnNames.append(value)
                    # Simple substring matches
                    elif 'Ao5' in value or 'Mo3' in value or 'Best' in value or value.startswith('Solve'):
                        columnNames.append(value)
                    # Substituted substring match #1
                    elif 'link' in value.lower():
                        columnNames.append('Link')
                    # Substituted substring match #2
                    elif value.startswith('Comment'):
                        columnNames.append('Comment')
                    # Columns to ignore
                    elif value.startswith('Pos') or value.startswith('Award') or value == '':
                        columnNames.append(None)
                    # Report any other columns
                    else:
                        print('WARNING: Unexpected field "{}" in {} ({})'.format(value, sheet.name, self.date))

                row = []
                for columnName in columnNames:
                    if columnName:
                        row.append(columnName)
                rows.append(row)

            # Process result row - must be after header on row 3
            elif rowNo >= 3:
                row = []
                for colNo in range(sheet.ncols):
                    if columnNames[colNo]:
                        fieldValue = sheet.cell(rowNo, colNo).value
                        if isinstance(fieldValue, str):
                            fieldValue = fieldValue.strip()
                        
                        # Standardise name
                        if columnNames[colNo] == 'Age':
                            try:
                                fieldValue = formatAge(interpretAge(fieldValue))
                            except:
                                print('ERROR: Age "{}" in {} ({})'.format(value, sheet.name, self.date))
                                raise

                        # Standardise result
                        elif 'Ao5' in columnNames[colNo] or 'Mo3' in columnNames[colNo] \
                                or 'Best' in columnNames[colNo] or 'Result' in columnNames[colNo] \
                                or columnNames[colNo].startswith('Solve'):
                            try:
                                fieldValue = formatResult(numSeconds(fieldValue), eventName)
                            except:
                                print('ERROR: Result "{}" in {} ({})'.format(value, sheet.name, self.date))
                                raise

                        # Standardise link
                        elif columnNames[colNo] == 'Link':
                            try:
                                fieldValue = formatFacebookLink(fieldValue)
                                if '/permalink/' not in fieldValue and '/videos/' not in fieldValue:
                                    fieldValue = ''
                            except:
                                print('ERROR: Link "{}" in {} ({})'.format(value, sheet.name, self.date))
                                raise

                        row.append(fieldValue)
                rows.append(row)

        fn = os.path.join(os.path.dirname(xlsx), eventName + '.csv')
        with open(fn, 'w') as outfile:
            csvWriter = csv.writer(outfile, quoting = csv.QUOTE_MINIMAL, lineterminator = os.linesep)
            csvWriter.writerows(rows)


    def processFormsSheet(self, sheet):
        '''Process a sheet which was filled in by Google Forms'''


    def processSheets(self, latest):
        '''Process a spreadsheet which has been downloaded from Google Sheets'''

        wb = open_workbook(self.xlsx)

        for sheet in wb.sheets():
            if sheet.name.lower() in events:
                self.processLegacySheet(sheet)
            elif sheet.name.startswith('Form Responses'):
                self.processFormsSheet(sheet)

## Main Code

Process all competitions

In [7]:
import glob
import time

pc1 = time.perf_counter()

In [8]:
# Process spreadsheets

datePattern = '[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]'
xlsxPattern = '*.xlsx'

persons = {}
competitions = {}

dataFiles = glob.glob(os.path.join(projdir, 'data', datePattern, xlsxPattern))
latest = False

for xlsx in dataFiles:
    spreadsheet = Spreadsheet(xlsx)
    if xlsx == dataFiles[-1]:
        latest = True
    spreadsheet.processSheets(latest)

In [9]:
pc2 = time.perf_counter()
print("Conversion completed in %0.2f seconds" % (pc2 - pc1))

Conversion completed in 13.11 seconds


## All Done!